In [1]:
import numpy as np
import pandas as pd
from os import listdir

In [2]:
def get_score_from_image(image_path, invert=0):
    # invert image from white font & black background to 
    # tesseract-enabled normal: black font & white background
    image = Image.open(image_path)
    if invert == 1:
        image = ImageOps.invert(image)
    text = pytesseract.image_to_string(image, config='--psm 6')
    try:
        res = int(text[0]), int(text[2])
    except:
        res = "a", "b"
    finally:
        return res

In [3]:
def create_time_series(InDirectory, player1="A", player2="B", who_starts=0, player1_id=1, player2_id=2, invert=0):
    # Walk through directory
    lst_images  = sorted(listdir(InDirectory))
    lst_results = []
    num_events  = len(lst_images)
    old_a       = 0
    old_b       = 0
    event       = 0
    old_time    = 0
    
    order  = "1100110011"
    order2 = "0011001100"
    if who_starts == 0:
        order = order2
    num_move = 0
    for i in range(150):
        current_image = lst_images[i]
        a,b = get_score_from_image(InDirectory+current_image, invert)
        if type(a) != int:
            a = old_a
        if type(b) != int:
            b = old_b
        if a != old_a:
            event = 1
        if b != old_b:
            event = 1
        if event == 1:
            print(current_image, a, b)
            num_move = num_move + 1
            if num_move < 11:
                who_serves = int(order[num_move-1])
            else:
                who_serves = (who_serves+1)%2
            time_stamp    = int(current_image[5:][:-4])
            time_to_score = time_stamp - old_time
            lst_entry     = [a, b, time_stamp, time_to_score, who_serves, player1_id, player2_id]
            lst_results.append(lst_entry)
            old_time = time_stamp
        old_a = a
        old_b = b
        event = 0
    columns = ["player1", "player2", "time_stamp", "time_to_score", "who_serves", "player1_id", "player2_id"]
    return pd.DataFrame(data=lst_results, columns=columns)

In [4]:
sorted(listdir("./../Data/WangHaoMaLin2008_halfed/"))

['image0000_lower.png',
 'image0000_lower_white.png',
 'image0000_upper.png',
 'image0000_upper_white.png',
 'image0001_lower.png',
 'image0001_lower_white.png',
 'image0001_upper.png',
 'image0001_upper_white.png',
 'image0002_lower.png',
 'image0002_lower_white.png',
 'image0002_upper.png',
 'image0002_upper_white.png',
 'image0003_lower.png',
 'image0003_lower_white.png',
 'image0003_upper.png',
 'image0003_upper_white.png',
 'image0004_lower.png',
 'image0004_lower_white.png',
 'image0004_upper.png',
 'image0004_upper_white.png',
 'image0005_lower.png',
 'image0005_lower_white.png',
 'image0005_upper.png',
 'image0005_upper_white.png',
 'image0006_lower.png',
 'image0006_lower_white.png',
 'image0006_upper.png',
 'image0006_upper_white.png',
 'image0007_lower.png',
 'image0007_lower_white.png',
 'image0007_upper.png',
 'image0007_upper_white.png',
 'image0008_lower.png',
 'image0008_lower_white.png',
 'image0008_upper.png',
 'image0008_upper_white.png',
 'image0009_lower.png',
 'im

In [407]:
player1_id = "A"
player2_id = "B"
who_starts = 0
InDirectory = "PNG/"
df = create_time_series(InDirectory, player1="A", player2="B", who_starts=0, player1_id=1, player2_id=2)

image005.png 1 0
image011.png 2 0
image016.png 3 0
image021.png 3 1
image026.png 4 1
image028.png 5 1
image031.png 5 2
image032.png 4 2
image037.png 4 3
image043.png 5 3
image046.png 3 3
image047.png 6 3
image052.png 7 3
image056.png 8 3
image062.png 9 3
image071.png 3 5
image074.png 9 5
image076.png 3 5
image095.png 0 1
image101.png 0 2
image109.png 0 3
image112.png 0 4
image116.png 1 4
image124.png 2 5
image128.png 3 5
image133.png 4 5
image139.png 3 5
image143.png 6 5
image148.png 6 6


In [409]:
df["player1"]

0     1
1     2
2     3
3     3
4     4
5     5
6     5
7     4
8     4
9     5
10    3
11    6
12    7
13    8
14    9
15    3
16    9
17    3
18    0
19    0
20    0
21    0
22    1
23    2
24    3
25    4
26    3
27    6
28    6
Name: player1, dtype: int64

In [358]:
def create_time_series(InDirectory, player1="A", player2="B", who_serves=0, player1_id=1, player2_id=2):
    """
    Create a time series for a point based game for two players, whose scores are saved in images.
    For that purpose provide a directory with images, whose filenames contains the measured time starting from 0.
    The time in the file names are used to infer the time for the time series.
    The images contain the picture for the scores of both players.
    For example, take the images from a youtube video, where there is a score display.
    The variable "who_serves" indicates, which player does start with serving.
    The rules for continuing the serving order are for table tennis (alternating every two rounds until 11 points,
    then alternate every round).
    """
    lst_images = sorted(listdir(InDirectory))
    lst_results = []
    num_events = len(lst_images)
    arr_events = np.zeros(num_events)
    old_timestamp = 0
    num_points = 0
    old_a  = 0
    old_b  = 0
    order  = "1100110011"
    order2 = "0011001100"
    if who_starts == 0:
        order = order2
    for i in range(num_events):
        file = lst_images[i]                        # Get file name
        a,b = get_score_from_image(InDirectory+file)# Get scores of both players
        if type(a) != int:
            a = old_a
        if type(b) != int:
            b = old_b
        player1 = a                                 # Score of player a
        player2 = b                                 # Score of player b
        time_stamp = int(file[5:][:-4])             # Use file name to extract time (dirty!)
        time_to_score = time_stamp-old_timestamp    # Time difference => Time of one move/round
        #if((a != old_a) or (b != old_b)):           # Condition: If there is a score, then do ...
        #    num_points = num_points+1               #            Count point
        #    if num_points < 11:                     #            Table tennis rule: Alternate serving every two rounds
        #        who_serves = int(order[num_points-1])
        #    else:                                   #            Start alternating serving every round
        #        who_serves = who_serves%2           #
        lst_entry = [player1, player2, time_stamp, time_to_score, who_serves, player1_id, player2_id]
        print(lst_entry)
        old_timestamp = time_stamp                  # Update time
        lst_results.append(lst_entry)               # Update entry for pd.DataFrame
        old_a = a
        old_b = b
    columns = ["player1", "player2", "time_stamp", "time_to_score", "who_serves", "player1_id", "player2_id"]
    return pd.DataFrame(data=lst_results, columns=columns)

<class 'str'>
<class 'str'>
a b
<class 'str'>
<class 'str'>
a b
<class 'str'>
<class 'str'>
a b
<class 'str'>
<class 'str'>
a b
<class 'str'>
<class 'str'>
a b
<class 'int'>
<class 'int'>
1 0
<class 'int'>
<class 'int'>
1 0
<class 'int'>
<class 'int'>
1 0
<class 'int'>
<class 'int'>
1 0
<class 'int'>
<class 'int'>
1 0
<class 'int'>
<class 'int'>
1 0
<class 'int'>
<class 'int'>
2 0
<class 'int'>
<class 'int'>
2 0
<class 'int'>
<class 'int'>
2 0
<class 'int'>
<class 'int'>
2 0
<class 'int'>
<class 'int'>
2 0
<class 'int'>
<class 'int'>
3 0
<class 'int'>
<class 'int'>
3 0
<class 'int'>
<class 'int'>
3 0
<class 'int'>
<class 'int'>
3 0


In [344]:
type(df)

NoneType

In [322]:
lst_images = sorted(listdir(InDirectory))
for i in lst_images:
    print(i)
    get_score_from_image("PNG/" + i)

image000.png
:

image001.png
:

image002.png
:

image003.png
7

image004.png
o
0

image005.png
1
0

image006.png
1
0

image007.png
1
0

image008.png
1
0

image009.png
1
0

image010.png
1
0

image011.png
2
0

image012.png
2
0

image013.png
2
0

image014.png
2
0

image015.png
2
0

image016.png
3
0

image017.png
3
0

image018.png
3
0

image019.png
3
0

image020.png
3
0

image021.png
3
1

image022.png
3
1

image023.png
3
1

image024.png
3
+

image025.png
3
1

image026.png
4
1

image027.png
4
1

image028.png
5
1

image029.png
5
1

image030.png
5
1

image031.png
5
2

image032.png
4
2

image033.png
4
2

image034.png
4
2

image035.png
4
2

image036.png
4
2

image037.png
4
3

image038.png
4
3

image039.png
4
3

image040.png
4
3

image041.png
4
3

image042.png
4
3

image043.png
5
3

image044.png
5
3

image045.png
5
3

image046.png
3
3

image047.png
6
3

image048.png
6
3

image049.png
6
3

image050.png
6
3

image051.png
T
3

image052.png
7
3

i

1
1

image429.png
1
1

image430.png
1
1

image431.png
1
1

image432.png
a
1

image433.png
a
1

image434.png
a
+

image435.png
a
+

image436.png
2
1

image437.png
2
1

image438.png
2
2

image439.png
2
2

image440.png
2
2

image441.png
2
2

image442.png
2
2

image443.png
2
3

image444.png
2
3

image445.png
2
3

image446.png
2
3

image447.png
2
3

image448.png
2
3

image449.png
2
3

image450.png
2
3

image451.png
3
3

image452.png
3
3

image453.png
3
3

image454.png
3
3

image455.png
3
3

image456.png
3
z=

image457.png
3
z=

image458.png
3
4

image459.png
3
4

image460.png
3
4

image461.png
3
4

image462.png
3
5

image463.png
3
5

image464.png
3
5

image465.png
3
5

image466.png
3
5

image467.png
3
5

image468.png
3
5

image469.png
=
5

image470.png
a
5

image471.png
4
5

image472.png
4
5

image473.png
4
5

image474.png
5
5

image475.png
3
5

image476.png
3
5

image477.png
3
5

image478.png
3
5

image479.png
3
5

image480.png
3
5

imag